## This notebook parses the HCC and RAF Improvement Opportunity  by Organization - Provider  

###  Added Excel Automation including Filtering, Styling, Security, Pre-filtering 

### Joe Eberle - 3/27/2022


In [1]:
# Import Libraries for Spreadsheet Automation 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import random
import math
import os
import glob
from openpyxl import load_workbook
from openpyxl.styles import colors
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle, Font, Border, Side, PatternFill, GradientFill, Alignment, Color, Fill
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows 
from openpyxl.formatting.rule import DataBarRule
from openpyxl.formatting.rule import ColorScaleRule
import time

In [2]:
# Configure the code and visual settings to have consistency 
path = 'c:/Users/josep/Documents/Kaleida/HCC/'
Path_to_Excel = 'c:/Users/josep/Documents/Kaleida/HCC/'
Path_to_Save_Excel = 'c:/Users/josep/Documents/Kaleida/HCC_ft/'

# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'
Path_to_Raw_Data = 'c:/Users/josep/Documents/Kaleida/'
Raw_Data_File_Name_1 = 'HCC and RAF Score Improvement Opportunity.xlsx'
Raw_Data_File_Name_2 = 'Organization_Provider_Summary.xlsx'
Report_Name_Extension ='_HCC_RAF'

print(Path_to_Raw_Data + Raw_Data_File_Name_1)
print(Path_to_Raw_Data + Raw_Data_File_Name_2)

LIGHT_BLUE = "C5D9F1"
YELLOW = "FFFF00"
GRAY = "E4DFEC"
DEEP_BLUE = "1072BA"
WHITE = "FFFFFF"

CONTENT_TAB_COLOR = DEEP_BLUE
USER_TAB_COLOR = YELLOW
BLANK_WHITE = WHITE 
HEADER_COLOR = LIGHT_BLUE  # Light Blue - Light Blue indicates relevant content for validation 
HEADER_COLOR_USER_ENTRY = YELLOW # Yellow - Yellow indicates a field for User Entry 
HEADER_COLOR_INFORMATIONAL_ONLY = GRAY # Gray - Indicates a field that maybe helpful but for information purposes only 

c:/Users/josep/Documents/Kaleida/HCC and RAF Score Improvement Opportunity.xlsx
c:/Users/josep/Documents/Kaleida/Organization_Provider_Summary.xlsx


In [3]:
# Read in the CSV SQL output for NCQA HEDIS Quality Measures 
Organizational_Data = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_1)

In [4]:
# Read in the EXCEL  SQL output for Organization Provider Summary 
Organization_Provider_Summary = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_2)

In [ ]:
# Filter to Measures that are analytics only , calculate Percent complete and round 
#Organizational_Quality['Measure_Completion_Percentage'] = ((Organizational_Quality['Total_Met_Count'] / Organizational_Quality['Total_Measure_Count']) * 100.0) 
#Organizational_Quality['Measure_Completion_Percentage'] = Organizational_Quality['Measure_Completion_Percentage'].round(2)


In [6]:
Organizational_Data.columns

Index(['Empi_ID', 'Condition_Display', 'Condition_Type', 'Is_Trumped',
       'Persistence_Suspected_Status', 'Suspected_Stratification',
       'Coefficient_Value', 'ICD_10_Code', 'ICD_10_Description',
       'Organization_Class', 'Organization_Name', 'Provider_Name',
       'Patient_Name', 'Age', 'Number_of_Conditions', 'Mara_Risk_Score',
       'Number_of_HCC', 'Actual_CMS_RAF_Score', 'Potential_CMS_RAF_Score',
       'Patient_ID'],
      dtype='object')

In [ ]:
# Filter to Measures that are analytics only , calculate Percent complete and round 
#Organizational_Quality = Organizational_Quality[(Organizational_Quality.Is_Registries_Measure == 'Analytics Only')]   
# Filter to Measures that are analytics only 
#Organizational_Quality['Is_Registries_Measure'].value_counts()


In [7]:
# simplify and Organize the organization quality Panels 
Organizational_Data = Organizational_Data[['Provider_Name','Patient_Name', 'Age',
       'Condition_Display', 'Condition_Type', 'Is_Trumped',
       'Persistence_Suspected_Status', 'Suspected_Stratification',
       'Coefficient_Value', 'ICD_10_Code', 'ICD_10_Description',
       'Organization_Class', 'Organization_Name',  
       'Number_of_Conditions', 'Mara_Risk_Score',
       'Number_of_HCC', 'Actual_CMS_RAF_Score', 'Potential_CMS_RAF_Score', 
       'Patient_ID',]]

#Organizational_Quality = Organizational_Quality[['Organization_Class', 'Organization_Name', 'Provider_Name',
#       'Patient_Name', 'Birth_Date', 'Age_Years', 'Age', 'Gender', 'Phone',
#       'Deceased', 'Deceased_Date', 'Number_of_Conditions', 'Mara_Risk_Score',
#      'Number_of_HCC', 'Actual_CMS_RAF_Score', 'Potential_CMS_RAF_Score',
#       'Age_at_Death', 'Attribution_Source_Type', 'Attribution_Record_Type',
#       'Attribution_Source', 'Patient_ID', 'Provider_ID', 'Plan_Name',
#       'Payer_Name', 'population_id', 'empi_id', 'hcc_year_id', 'hcc_alias',
#       'hcc_model_number', 'hcc_Description', 'hcc_coeff', 'ICD_10_code',
#       'ICD_10_Description', 'Chronic_Dx_Indicator', 'HCC_Key', 'redoc_ind',
#       'redoc_clinical_ind', 'redoc_hcc_severity', 'redoc_hcc_change',
#       'current_year_suppression_ind', 'attending_provider_last_name_first',
#       'validation_type', 'effective_date', 'effective_date_id']]


In [8]:
# Optional Verify the input file structures 
#Organizational_Quality.info()
#Organization_Provider_Summary.head()
Organizational_Data.columns 

Index(['Provider_Name', 'Patient_Name', 'Age', 'Condition_Display',
       'Condition_Type', 'Is_Trumped', 'Persistence_Suspected_Status',
       'Suspected_Stratification', 'Coefficient_Value', 'ICD_10_Code',
       'ICD_10_Description', 'Organization_Class', 'Organization_Name',
       'Number_of_Conditions', 'Mara_Risk_Score', 'Number_of_HCC',
       'Actual_CMS_RAF_Score', 'Potential_CMS_RAF_Score', 'Patient_ID'],
      dtype='object')

In [9]:
Color_Scale_Rule = ColorScaleRule(start_type="min",start_color="FF0000",end_type="max",end_color="99FF33")
Data_Bar_Rule = DataBarRule(start_type="min", end_type="max", color="00CC00")

In [10]:
# The following function creates patient panels based upon organization name 
def create_organization_panel(org_name):
    Organization_panel = Organizational_Data[(Organizational_Data.Organization_Name == org_name)]    
    patient_rows = len(Organization_panel) - 1 
    if patient_rows > 2:
        spread_name = Organization_panel['Organization_Name'].iloc[0]
        if spread_name == '':
            spread_name = 'No_ORG_NAME_'  
        spread_name = spread_name.replace('/','')  
       
        spath =  Path_to_Excel

        csv_name = spath + spread_name.replace(' ','_') + Report_Name_Extension + '.csv'        
        spread_name = spath + spread_name.replace(' ','_') + Report_Name_Extension + '.xlsx'
        spread_name = spread_name.replace(' ','')
        csv_name = csv_name.replace(' ','')        
        Organization_panel.to_excel(spread_name,sheet_name='HCC Opportunity', index = False)  
        Organization_panel.to_csv(csv_name, index = False)  
        print(spread_name)
        return org_name

In [11]:
Org_names = Organizational_Data.Organization_Name.unique()
for org in Org_names:
    create_organization_panel(org)

c:/Users/josep/Documents/Kaleida/HCC/GPPC_PCP_Lancaster_Transit_HCC_RAF.xlsx


In [12]:
# Add the Provider Summary Excel Tab  
path = 'c:/Users/josep/Documents/Kaleida/HCC/'
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))

for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
  
    org_name =  files[x].replace(' HCC RAF','')      
    org_name =  org_name.replace('_HCC_RAF','')   
     
    org_name =  org_name.replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    print (file_path)
    Organization_panel = Organization_Provider_Summary[(Organization_Provider_Summary.Organization_Name == org_name)]  
    Organization_panel = Organization_panel.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Organization_panel.to_excel(writer,sheet_name='Provider Summary', index = False)
    writer.save()
    writer.close()

GPPC PCP Lancaster Transit
c:/Users/josep/Documents/Kaleida/HCC/GPPC_PCP_Lancaster_Transit_HCC_RAF.xlsx


In [13]:
# The following function creates patient panels based upon organization name 
def create_sheet_one(work_book):
   
    ws1 = work_book.active
    ws1.title = "HCC and RAF Opprtunity"  
    ws1.sheet_properties.tabColor = CONTENT_TAB_COLOR
    
# Format the Header as bold blue centered wordwrap     
    max_cols = ws1.max_column + 1 
    for col in range(1,max_cols):
        ws1[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws1[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws1[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)  
    
# Work Sheet 1 - Set the default column width to a min width 15 
    for col in range(1,max_cols):
        ws1.column_dimensions[get_column_letter(col)].width = 10
    ws1.column_dimensions['A'].width = 20
    ws1.column_dimensions['F'].width = 40        
    ws1.column_dimensions['H'].width = 40
    ws1.column_dimensions['K'].width = 40
    
    ws1.row_dimensions[1].height = 75   

# Replaces all underscores in header with two spaces 
    for col in range(1,max_cols):
        ws1[get_column_letter(col) + '1'].value = ws1[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws1.max_column) \
    + str(ws1.max_row)
    ws1.auto_filter.ref = FullRange    

  
    ws1.freeze_panes = 'A2'
#    ws1.conditional_formatting.add("F1:F" + str(ws1.max_row), Color_Scale_Rule )
        
    return work_book  

In [ ]:
# The following function formats provider summary 
def create_sheet_two(work_book):

    sheets = work_book.sheetnames
   
    ws2 = work_book[sheets[1]]
   # ws = wb[sheets[n]]
    ws2.title = "Provider Summary"
    ws2.sheet_properties.tabColor = CONTENT_TAB_COLOR
     
# Format the Header as bold blue centered wordwrap 
    ws2.row_dimensions[1].height = 60 
    max_cols = ws2.max_column + 1 
    for col in range(1,max_cols):
        ws2[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws2[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws2[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True) 
# Replaces all underscores in header with two spaces         
        ws2[get_column_letter(col) + '1'].value = ws2[get_column_letter(col) + '1'].value.replace('_','  ')        
    
# Work Sheet 2 - Set Column Widths and Header Hieght
    ws2.column_dimensions['A'].width = 30
    ws2.column_dimensions['B'].width = 30
    ws2.column_dimensions['C'].width = 30
    ws2.column_dimensions['D'].width = 30
    ws2.column_dimensions['E'].width = 30
    ws2.column_dimensions['F'].width = 30
    
    

# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws2.max_column) \
    + str(ws2.max_row)
    ws2.auto_filter.ref = FullRange    

    return work_book  

In [ ]:
def create_sheet_feedback(work_book):
    
    # Create The Tab 
    ws_feedback = work_book.create_sheet("Feedback")
    ws_feedback.title = "Feedback or Issues" 
    ws_feedback.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_feedback[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_feedback[get_column_letter(col) + str(row)].font = fontStyle
            ws_feedback[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
    
    
    # Create The Title 
    ws_feedback.merge_cells('A1:B1')
    ws_feedback.row_dimensions[1].height = 20
    top_left_cell = ws_feedback['A1']
    top_left_cell.value = "Feedback or Issues"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws_feedback['A2'] = 'User Name'
    ws_feedback['B2'] = 'Issue or Feedback'
  
    for col in range(1,3):
        ws_feedback[get_column_letter(col) + '2'].font = Font(size = "12", bold=True ) 
        ws_feedback[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        ws_feedback[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor=HEADER_COLOR)
    
    ws_feedback.column_dimensions['A'].width = 30
    ws_feedback.column_dimensions['B'].width = 120 

    ws_feedback.row_dimensions[2].height = 30
 
   
    return work_book  
 



In [ ]:
def create_instruction_sheet(work_book):
    
    ws_instruction  = work_book.create_sheet("Instructions")
    ws_instruction.title = "Instructions" 
    ws_instruction.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_instruction[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_instruction[get_column_letter(col) + str(row)].font = fontStyle
            ws_instruction[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
            
    for row in range(1,25):
            ws_instruction.row_dimensions[row].height = 20         
            
    ws_instruction.column_dimensions['A'].width = 3
    ws_instruction.column_dimensions['B'].width = 3
    ws_instruction.column_dimensions['C'].width = 80            
    
    
    ws_instruction.merge_cells('A1:F1')
    top_left_cell = ws_instruction['A1']
    top_left_cell.value = "Instructions for Validation"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    fontStyle = Font(size = "14", bold = "True")
    ws_instruction.merge_cells('A3:F3')
    ws_instruction['A3'].font = fontStyle
    ws_instruction['A3'] = 'Spreadsheet instructions:'


    ws_instruction['B4'] = 1
    ws_instruction['B5'] = 2 
    
    ws_instruction['C4'] = 'Use Filter column A  \'Provider Name \' to select the provider you want'
    ws_instruction['C5'] = 'If any issues or problems are identified, please enter them on the \'Feedback\' tab. '       
    
    return work_book  

In [ ]:
# Automate the worksheet 
def create_work_book(work_book):
    create_sheet_one(wb)
    create_sheet_two(wb)
    create_instruction_sheet(wb) 
    create_sheet_feedback(wb)    
    return work_book      

In [ ]:
Path_to_Excel = 'c:/Users/josep/Documents/Kaleida/HCC/'
Path_to_Save_Excel = 'c:/Users/josep/Documents/Kaleida/HCC_ft/'
extension = 'xlsx'
os.chdir(Path_to_Excel)
files = glob.glob('*.{}'.format(extension))

for x in range(len(files)): 
#    print(files[x]) 
#    print(Path_to_Excel + files[x])     
    wb = load_workbook(Path_to_Excel + files[x])
    org_name =  files[x].replace('.xlsx','')
    create_work_book(wb)
    wb.save(Path_to_Save_Excel + files[x])
    print(Path_to_Save_Excel + files[x])   
